In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c fake-news

mkdir: cannot create directory ‘/root/.kaggle’: File exists
 86% 32.0M/37.0M [00:00<00:00, 83.7MB/s]
100% 37.0M/37.0M [00:00<00:00, 107MB/s] 
 64% 6.00M/9.42M [00:00<00:00, 58.9MB/s]
100% 9.42M/9.42M [00:00<00:00, 60.0MB/s]
  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 13.3MB/s]


In [4]:
! unzip train.csv.zip
! unzip test.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                


In [5]:
import pandas as pd
df = pd.read_csv('/content/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [7]:
df.shape

(20800, 5)

In [12]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [15]:
df['author'].unique().shape

(4202,)

In [17]:
df.drop(['author'],axis=1,inplace=True)
df.head()

,id,title,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1


In [18]:
df.dropna(inplace=True)
df.shape

(20203, 4)

Using count vectorizer

In [20]:
df.title.replace("[^a-zA-Z]"," ",regex=True, inplace=True)
df.text.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

In [21]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [23]:
lem = WordNetLemmatizer()
def convert(message):
  message = re.sub('[^a-zA-Z]',' ',message)
  words = nltk.word_tokenize(message)
  words = [lem.lemmatize(word) for word in words if not word in set(stopwords.words('english'))]
  return ' '.join(words)

In [24]:
df2 = pd.DataFrame()
df2['title'] = df.title.apply(lambda x : convert(str(x)))
df2['text'] = df.text.apply(lambda x : convert(str(x)))
df2.head()

,title,text
0,House Dem Aide We Didn Even See Comey Letter U...,House Dem Aide We Didn Even See Comey Letter U...
1,FLYNN Hillary Clinton Big Woman Campus Breitbart,Ever get feeling life circle roundabout rather...
2,Why Truth Might Get You Fired,Why Truth Might Get You Fired October The tens...
3,Civilians Killed In Single US Airstrike Have B...,Videos Civilians Killed In Single US Airstrike...
4,Iranian woman jailed fictional unpublished sto...,Print An Iranian woman sentenced six year pris...


In [25]:
df2['title'] = df2.title.apply(lambda x : str(x).lower())
df2['text'] = df2.text.apply(lambda x : str(x).lower())
df2.head()

,title,text
0,house dem aide we didn even see comey letter u...,house dem aide we didn even see comey letter u...
1,flynn hillary clinton big woman campus breitbart,ever get feeling life circle roundabout rather...
2,why truth might get you fired,why truth might get you fired october the tens...
3,civilians killed in single us airstrike have b...,videos civilians killed in single us airstrike...
4,iranian woman jailed fictional unpublished sto...,print an iranian woman sentenced six year pris...


In [26]:
from sklearn.feature_extraction.text import CountVectorizer

In [27]:
countvector=CountVectorizer(min_df=2)
traindataset=countvector.fit_transform(df2['text'])

In [28]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
randomclassifier=RandomForestClassifier()
randomclassifier.fit(traindataset,df['label'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [31]:
traindataset.shape

(20203, 72905)

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(traindataset,df['label'] , test_size=0.2, random_state=42)

In [35]:
X_train.shape

(16162, 72905)

In [36]:
randomclassifier=RandomForestClassifier()
randomclassifier.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [37]:
randomclassifier.score(X_test,y_test)

0.9287305122494433